In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import glob
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import io
import requests
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import glob

In [3]:
# fix random seed for reproducibility
np.random.seed(7)

In [4]:
allfiles = glob.glob('dataset/*.csv')
df = pd.concat((pd.read_csv(f) for f in allfiles))

C:\Users\Susmit\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  
C:\Users\Susmit\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  


In [5]:
df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.16-199.244.48.55-41936-443-6,192.168.10.16,41936,199.244.48.55,443,6,07/07/2017 3:30,143347,47,60,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.16-54.210.195.63-42970-80-6,192.168.10.16,42970,54.210.195.63,80,6,07/07/2017 3:30,50905,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.16-199.244.48.55-41944-443-6,192.168.10.16,41944,199.244.48.55,443,6,07/07/2017 3:30,143899,46,58,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.3-192.168.10.17-53-12886-17,192.168.10.17,12886,192.168.10.3,53,17,07/07/2017 3:30,313,2,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.16-199.244.48.55-41942-443-6,192.168.10.16,41942,199.244.48.55,443,6,07/07/2017 3:30,142605,45,58,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [6]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1967674 entries, 0 to 445908
Data columns (total 85 columns):
Flow ID                         object
 Source IP                      object
 Source Port                    int64
 Destination IP                 object
 Destination Port               int64
 Protocol                       int64
 Timestamp                      object
 Flow Duration                  int64
 Total Fwd Packets              int64
 Total Backward Packets         int64
Total Length of Fwd Packets     float64
 Total Length of Bwd Packets    float64
 Fwd Packet Length Max          float64
 Fwd Packet Length Min          float64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max           float64
 Bwd Packet Length Min          float64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    object
 Flow Packets/s                 object
 Flow IAT Mean            

In [7]:
dataset = df.values
y = dataset[:,84]

In [8]:
df[['Source IP1','Source IP2','Source IP3', 'Source IP4']] = df[' Source IP'].str.split('.', expand=True)
df[['Destination IP1','Destination IP2','Destination IP3', 'Destination IP4']] = df[' Destination IP'].str.split('.', expand=True)

df = df.drop(' Source IP', axis=1)
df = df.drop(' Destination IP', axis=1)

df = df.drop('Flow ID', axis=1)
df = df.drop(' Timestamp', axis=1)
df = df.drop(' Flow Packets/s', axis=1)
df = df.drop('Flow Bytes/s', axis=1)

df = df.drop('Fwd Avg Bytes/Bulk', axis=1)
df = df.drop(' Fwd Avg Packets/Bulk', axis=1)
df = df.drop(' Fwd Avg Bulk Rate', axis=1)
df = df.drop(' Bwd Avg Bytes/Bulk', axis=1)
df = df.drop(' Bwd Avg Packets/Bulk', axis=1)
df = df.drop('Bwd Avg Bulk Rate', axis=1)

df = df.drop(' Bwd PSH Flags', axis=1)
df = df.drop(' Bwd URG Flags', axis=1)

df.head()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,Idle Min,Label,Source IP1,Source IP2,Source IP3,Source IP4,Destination IP1,Destination IP2,Destination IP3,Destination IP4
0,41936,443,6,143347,47,60,1325.0,108751.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
1,42970,80,6,50905,1,1,0.0,0.0,0.0,0.0,...,0.0,BENIGN,192,168,10,16,54,210,195,63
2,41944,443,6,143899,46,58,1325.0,110185.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
3,12886,53,17,313,2,2,90.0,206.0,45.0,45.0,...,0.0,BENIGN,192,168,10,17,192,168,10,3
4,41942,443,6,142605,45,58,1325.0,108751.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55


In [9]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1967674 entries, 0 to 445908
Data columns (total 79 columns):
 Source Port                    int64
 Destination Port               int64
 Protocol                       int64
 Flow Duration                  int64
 Total Fwd Packets              int64
 Total Backward Packets         int64
Total Length of Fwd Packets     float64
 Total Length of Bwd Packets    float64
 Fwd Packet Length Max          float64
 Fwd Packet Length Min          float64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max           float64
 Bwd Packet Length Min          float64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                   float64
 Flow IAT Min                   float64
Fwd IAT Total                   float64
 Fwd IAT Mean                   float64
 Fwd IAT Std        

In [10]:
df[['Source IP1','Source IP2','Source IP3', 'Source IP4', 'Destination IP1','Destination IP2','Destination IP3', 'Destination IP4']] = df[['Source IP1','Source IP2','Source IP3', 'Source IP4', 'Destination IP1','Destination IP2','Destination IP3', 'Destination IP4']].apply(pd.to_numeric)

In [11]:
df.apply(pd.to_numeric, errors='ignore')

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,Idle Min,Label,Source IP1,Source IP2,Source IP3,Source IP4,Destination IP1,Destination IP2,Destination IP3,Destination IP4
0,41936,443,6,143347,47,60,1325.0,108751.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
1,42970,80,6,50905,1,1,0.0,0.0,0.0,0.0,...,0.0,BENIGN,192,168,10,16,54,210,195,63
2,41944,443,6,143899,46,58,1325.0,110185.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
3,12886,53,17,313,2,2,90.0,206.0,45.0,45.0,...,0.0,BENIGN,192,168,10,17,192,168,10,3
4,41942,443,6,142605,45,58,1325.0,108751.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
5,33063,53,17,253,2,2,90.0,122.0,45.0,45.0,...,0.0,BENIGN,192,168,10,17,192,168,10,3
6,41940,443,6,142499,46,53,1313.0,100063.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
7,41938,443,6,23828,27,31,162.0,59368.0,6.0,6.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
8,41946,443,6,119090,23,28,1169.0,45025.0,570.0,0.0,...,0.0,BENIGN,192,168,10,16,199,244,48,55
9,123,123,17,63021198,2,2,96.0,96.0,48.0,48.0,...,63000000.0,BENIGN,192,168,10,25,17,253,14,125


In [12]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1967674 entries, 0 to 445908
Data columns (total 79 columns):
 Source Port                    int64
 Destination Port               int64
 Protocol                       int64
 Flow Duration                  int64
 Total Fwd Packets              int64
 Total Backward Packets         int64
Total Length of Fwd Packets     float64
 Total Length of Bwd Packets    float64
 Fwd Packet Length Max          float64
 Fwd Packet Length Min          float64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max           float64
 Bwd Packet Length Min          float64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                   float64
 Flow IAT Min                   float64
Fwd IAT Total                   float64
 Fwd IAT Mean                   float64
 Fwd IAT Std        

In [13]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
print(encoded_Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y)

[0 0 0 ..., 0 0 0]
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [22]:
# np.savetxt("Encoded.csv", dummy_y, delimiter=",")

In [14]:
df = df.drop(' Label', axis = 1)

In [15]:
df.info(memory_usage= "deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1967674 entries, 0 to 445908
Data columns (total 78 columns):
 Source Port                    int64
 Destination Port               int64
 Protocol                       int64
 Flow Duration                  int64
 Total Fwd Packets              int64
 Total Backward Packets         int64
Total Length of Fwd Packets     float64
 Total Length of Bwd Packets    float64
 Fwd Packet Length Max          float64
 Fwd Packet Length Min          float64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max           float64
 Bwd Packet Length Min          float64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                   float64
 Flow IAT Min                   float64
Fwd IAT Total                   float64
 Fwd IAT Mean                   float64
 Fwd IAT Std        

In [24]:
dataset = df.values

x = dataset[:,0:78]
x.shape

(1967674, 78)

In [34]:
x_train, x_test, y_train, y_test = train_test_split(
    x, dummy_y, test_size=0.25, random_state=42)

In [30]:
dummy_y.shape[1]

7

In [35]:
# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(dummy_y.shape[1],activation='softmax'))

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

In [38]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Train on 1475755 samples, validate on 491919 samples
Epoch 1/1000
 - 222s - loss: 1.8419 - acc: 0.8853 - val_loss: 1.7798 - val_acc: 0.8896
Epoch 2/1000
 - 213s - loss: 1.7724 - acc: 0.8900 - val_loss: 1.7798 - val_acc: 0.8896
Epoch 3/1000
 - 213s - loss: 1.7724 - acc: 0.8900 - val_loss: 1.7798 - val_acc: 0.8896
Epoch 4/1000
 - 216s - loss: 1.7724 - acc: 0.8900 - val_loss: 1.7798 - val_acc: 0.8896
Epoch 5/1000
 - 213s - loss: 1.7724 - acc: 0.8900 - val_loss: 1.7798 - val_acc: 0.8896
Epoch 6/1000
 - 210s - loss: 1.7724 - acc: 0.8900 - val_loss: 1.7798 - val_acc: 0.8896
Epoch 00006: early stopping


In [39]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.8895753162614171


In [40]:
from keras.models import model_from_json

In [41]:
# save model to JSON
model_json = model.to_json()
with open("model_mk2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_mk2.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# load json and create model
json_file = open('model_mk3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_mk3.h5")
print("Loaded model from disk")